# Exercício 05 — Data Wrangling
**Disciplina:** Machine Learning - Técnicas e Cases
**Professor:** Raphael Cobé
**Aluno:** Rafael de Toledo Risso
**Dataset:** MovieLens 100k


## Objetivo
Aplicar técnicas de data wrangling na base MovieLens 100k, preparando os dados para análise, tratando valores ausentes, criando novas colunas e extraindo insights.

In [ ]:
# Carregar os dados de filmes e avaliações
import pandas as pd
import matplotlib.pyplot as plt

movies = pd.read_csv('./ml-100k/u.item', sep='|', encoding='latin-1', header=None,
    names=['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url',
           'unknown', 'Action', 'Adventure', 'Animation', "Children's", 'Comedy',
           'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
           'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'])
ratings = pd.read_csv('./ml-100k/u.data', sep='\t', names=['user_id', 'movie_id', 'rating', 'timestamp'])

In [ ]:
# Verificar e tratar valores ausentes na data de lançamento
missing_dates = movies['release_date'].isna().sum() + (movies['release_date'] == '').sum()
movies['release_date'] = pd.to_datetime(movies['release_date'], errors='coerce')
print(f'Filmes com data de lançamento ausente: {missing_dates}')

In [ ]:
# Criar coluna de década do filme
movies['decade'] = (movies['release_date'].dt.year // 10 * 10).astype('Int64')

In [ ]:
# Filtrar apenas filmes do gênero Drama
drama_movies = movies[movies['Drama'] == 1]
print(f'Quantidade de filmes Drama: {drama_movies.shape[0]}')

In [ ]:
# Juntar notas dos filmes Drama
drama_ratings = pd.merge(drama_movies[['movie_id', 'title', 'release_date', 'decade']], ratings, on='movie_id', how='inner')

In [ ]:
# Calcular média de avaliações por década para dramas
drama_decade_stats = drama_ratings.groupby('decade')['rating'].agg(['mean', 'count']).reset_index()
print(drama_decade_stats)

In [ ]:
# Gráfico das médias por década
plt.figure(figsize=(8,4))
plt.bar(drama_decade_stats['decade'].astype(str), drama_decade_stats['mean'])
plt.xlabel('Década')
plt.ylabel('Média das Avaliações')
plt.title('Média das Avaliações de Filmes Drama por Década')
plt.tight_layout()
plt.show()

## Conclusão
- Realizei a limpeza dos dados, convertendo datas e identificando apenas um filme sem data de lançamento.
- Criei uma coluna para a década do filme, facilitando a análise histórica.
- Ao focar no gênero Drama, percebi que esse é um dos mais representativos do dataset.
- O gráfico revelou como a média das avaliações para filmes de Drama varia ao longo das décadas, com décadas como 1940 e 1970 apresentando médias mais altas.
- Esse tipo de análise é fundamental para entender tendências de gosto do público ao longo do tempo.